In [1]:
import cv2
import mediapipe as mp

cap = cv2.VideoCapture(0)

wCam, hCam = 640, 480

cap.set(3, wCam)
cap.set(4, hCam)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

hands = mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5)

In [2]:
def get_squared_distance(point1, point2):
    return (point1.x - point2.x) ** 2 + (point1.y - point2.y) ** 2

In [12]:
def check_constraints(hand_landmarks):
    if not hand_landmarks:
        return False
    landmarks = hand_landmarks[0].landmark
    fingers = [landmarks[8], landmarks[12], landmarks[16]]
    pinky_tip = landmarks[20]
    pinky_mcp = landmarks[17]
    are_fingers_together = all(get_squared_distance(finger, landmarks[4]) < 0.007 for finger in fingers)
    is_thumb_lowest = all(finger.y < landmarks[4].y for finger in fingers)
    print(pinky_tip.x - pinky_mcp.x, pinky_tip.y < pinky_mcp.y)
    is_pinky_straight_up = abs(pinky_tip.x - pinky_mcp.x) < 0.1 and pinky_tip.y < pinky_mcp.y
    
    return are_fingers_together and is_thumb_lowest and is_pinky_straight_up

In [4]:
def draw_landmarks(image, hand_landmarks):
    if not hand_landmarks:
        return
    for landmarks in hand_landmarks:
        mp_drawing.draw_landmarks(image, landmarks, mp_hands.HAND_CONNECTIONS)

In [ ]:
while True:
    success, image = cap.read()

    image = cv2.flip(image, 1)
    results = hands.process(image)
    hand_landmarks = results.multi_hand_landmarks
    draw_landmarks(image, hand_landmarks)
    if check_constraints(hand_landmarks):
        cv2.putText(image, "+", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Output", image)

    key = cv2.waitKey(1)
    if key == 32:
        break

cv2.destroyAllWindows()